In [2]:
# Import the Pandas package
import pandas as pd
import numpy as np
import re 
import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings

### Utilities

In [9]:
def clean_text(string,replace='_'):
    regex = '[^A-Za-z0-9]+'
    str1 = re.sub(regex, replace, string)
    regex = '[^A-Za-z_]+'
    return re.sub(regex, '', str1)

def load_files(year):
    temp_df = pd.read_excel(file_str.format(year), header = 3)
    column_list = temp_df.columns.to_list()
    if('Population' in column_list or 'population' in column_list):
        return temp_df.rename(columns = lambda col: clean_text(col.lower().strip()))
    else: 
        return pd.read_excel(file_str.format(year), header = 4).rename(columns = lambda col: clean_text(col.lower().strip()))
    
    
def remove_digits(string):
    return re.sub('\d', '', string)


def replace_emdash(string):
    return re.sub(u'\u2014','-',string)

def replace_all_special(string):
    return re.sub('[^A-Za-z0-9]+', '', string)

def replace_msa(string):
    string= string.replace('M.S.A.,','')
    string= string.replace('M.S.A.','')
    string= string.replace('M.S.A','')
    return string.strip()

def split_name(string):
    split = string.split(',')
    city= split[0].strip()
    state = split[1].strip()
    
    city_split=city.split('-')
    state_split=state.split('-')
    return city_split[0]+', '+state_split[0]

def clean_string(col):
    return col.apply(remove_digits).apply(replace_emdash).apply(replace_msa).apply(split_name).apply(replace_all_special)

### Load the Data Files

In [10]:
file_str = "data/crime_data/FBI_Crime_{}.xls"
dfs = []
for  i in range(2000,2018):
    dfs.append(load_files(i))

### Load and Select Rate per 1000 entries (Smh...)

In [11]:
clean_dfs = []
year = 2000
for i in range(len(dfs)):
    temp_df = dfs[i].copy()
    if 'counties_principal_cities' in temp_df.columns.to_list():
        temp_df[['metropolitan_statistical_area','counties_principal_cities']] = temp_df[['metropolitan_statistical_area','counties_principal_cities']].fillna('')
        index_to_drop = temp_df[temp_df['counties_principal_cities'].str.contains('per 100,000')].index.values[-1]+1
        temp_df = temp_df.drop(list(range(index_to_drop,temp_df.shape[0])),axis=0)
        values = temp_df.loc[temp_df['counties_principal_cities'].str.contains("per 100,000")].reset_index()
        names = temp_df.loc[temp_df['metropolitan_statistical_area'].str.contains("M.S.A") | 
                   temp_df['metropolitan_statistical_area'].str.contains("M.D.") & 
                    ~temp_df['metropolitan_statistical_area'].str.contains("Includes")].reset_index()
        print(values.shape,names.shape)
        columns = names.columns.to_list()[4:]
        names.loc[:,columns] = values[columns]
        if('unnamed_' in names.columns.to_list()):
            names.drop(columns = ["index",'unnamed_','counties_principal_cities'],inplace=True)
        else:
            names.drop(columns = ["index","counties_principal_cities"],inplace=True)
        names['year'] = str(year+i)
        clean_dfs.append(names)
    else:
        #find the asterisks and get rid of them
        index_to_drop = temp_df.loc[temp_df['metropolitan_statistical_area'].isnull()].index.values[0]
        temp_df = temp_df.drop(list(range(index_to_drop,temp_df.shape[0])),axis=0)
        # get the rows that contain the Rate Per 100,000
        # There extra rows with M.D for each large city
        values = temp_df.loc[temp_df['metropolitan_statistical_area'].str.contains(" per 100,000")].reset_index()
        names = temp_df.loc[temp_df['metropolitan_statistical_area'].str.contains("M.S.A") | 
                   temp_df['metropolitan_statistical_area'].str.contains("M.D.") & 
                    ~temp_df['metropolitan_statistical_area'].str.contains("Includes")].reset_index()
        # print validation
        print(values.shape,names.shape)
        columns = names.columns.to_list()
        names.loc[:,columns[3:]] = values[columns[3:]]
        names.drop(columns = ["index",'unnamed_'],inplace=True)
        names['year'] = str(year+i)
        clean_dfs.append(names)
        

(268, 16) (268, 16)
(280, 16) (280, 16)
(288, 16) (288, 16)
(333, 15) (333, 15)
(347, 15) (347, 15)
(358, 16) (358, 16)
(357, 16) (357, 16)
(364, 13) (364, 13)
(357, 13) (357, 13)
(378, 13) (378, 13)
(368, 13) (368, 13)
(376, 13) (376, 13)
(389, 13) (389, 13)
(384, 13) (384, 13)
(377, 13) (377, 13)
(378, 13) (378, 13)
(386, 13) (386, 13)
(394, 13) (394, 13)


### We are only concerned with metropolitan areas so we take only those rows with M.S.A
The areas we are getting rid of look like this

In [12]:
clean_dfs[-1][~clean_dfs[-1]['metropolitan_statistical_area'].str.contains("M.S.A")].head()

,metropolitan_statistical_area,population,violent_crime,murder_and_nonnegligent_manslaughter,rape,robbery,aggravated_assault,property_crime,burglary,larceny_theft,motor_vehicle_theft,year
41,"Boston, MA M.D.",2014011,432.3,3.8,35.4,104.4,288.7,1538.5,221.6,1196.7,120.1,2017
42,"Cambridge-Newton-Framingham, MA M.D.",2390681,226.9,1.8,18.8,41.5,164.8,1132.5,173.8,861.5,97.3,2017
43,"Rockingham County-Strafford County, NH M.D.",434337,148.5,1.2,40.5,22.6,84.3,1210.4,140.0,1017.2,53.2,2017
67,"Chicago-Naperville-Arlington Heights, IL M.D.",7313953,515.1,10.7,40.0,199.3,265,2137.2,307.5,1614.5,215.3,2017
68,"Elgin, IL M.D.",639123,182.9,1.7,31.9,34.6,114.7,1298,158.3,1085.2,54.4,2017


### Clean the names

In [13]:
temp_df[['metropolitan_statistical_area','counties_principal_cities']] = temp_df[['metropolitan_statistical_area','counties_principal_cities']].fillna('')
index_to_drop = temp_df[temp_df['counties_principal_cities'].str.contains('per 100,000')].index.values[-1]+1

In [14]:
for i in range(len(clean_dfs)):
    temp_df = clean_dfs[i]
    temp_df = temp_df[temp_df['metropolitan_statistical_area'].str.contains("M.S.A")]
    temp_df['metropolitan_statistical_area'] = clean_string(temp_df['metropolitan_statistical_area'])
    clean_dfs[i] = temp_df

In [17]:
merged_df = pd.concat(clean_dfs).rename(columns={'metropolitan_statistical_area':'city_fbi'})

In [18]:
merged_df.to_csv('CleanData/FBI_Crime.csv',index=False)

## Let's Test

In [19]:
test_df = pd.read_csv('CleanData/FBI_Crime.csv')
test_df

,aggravated_assault,arson,burglary,crime_index,crime_index_total,forcible_rape,larceny_theft,city_fbi,modified_crime_index,modified_crime_index_total,motor_vehicle_theft,murder_and_non_negligent_man_slaughter,murder_and_nonnegligent_manslaughter,population,property_crime,rape,robbery,violent_crime,year
0,217.4,,819.4,NaN,4035.7,40.8,2683.4,AbileneTX,NaN,NaN,189.9,0.8,NaN,127413,3692.7,NaN,84.0,343,2000
1,263.3,NaN,1310.9,NaN,5490.8,30.8,3398.9,AlbanyGA,NaN,,283.6,5.7,NaN,123424,4993.4,NaN,197.7,497.5,2000
2,190.1,NaN,601.4,NaN,3191.2,21.0,2140.6,AlbanyNY,NaN,NaN,140.1,2.5,NaN,906739,2882.1,NaN,95.5,309.1,2000
3,641.5,NaN,1333.1,NaN,7038.8,44.5,4012.0,AlbuquerqueNM,NaN,,748.0,7.7,NaN,709724,6093.1,NaN,251.9,945.7,2000
4,473.8,NaN,1876.7,NaN,6900.4,50.9,4083.0,AlexandriaLA,NaN,,250.0,8.5,NaN,129586,6209.8,NaN,157.4,690.7,2000
5,168.7,NaN,510.5,NaN,3021.6,17.8,2020.9,AllentownPA,NaN,NaN,205.6,2.8,NaN,633148,2737.0,NaN,95.2,284.6,2000
6,145.9,NaN,537.8,NaN,2666.6,21.8,1758.7,AltoonaPA,NaN,,130.9,1.5,NaN,132942,2427.4,NaN,70.0,239.2,2000
7,519.6,NaN,1074.6,NaN,6773.8,50.7,4593.7,AmarilloTX,NaN,,408.1,5.5,NaN,217100,6076.5,NaN,121.6,697.4,2000
8,372.9,NaN,587.6,NaN,4931.4,74.7,3372.6,AnchorageAK,NaN,,387.1,3.8,NaN,260900,4347.3,NaN,132.6,584.1,2000
9,166.4,NaN,521.7,NaN,3055.5,34.9,2053.7,AnnArborMI,NaN,,219.5,1.8,NaN,558168,2794.9,NaN,57.5,260.7,2000
